In [3]:
import numpy as np
from copy import deepcopy
from clarkewright import *  
from vendorpickup import * 

### Step 1

In [4]:
#초기 거리 값  
# full_mat is travel_time_matrix of from vendor to vendor 
f = [0,1.6,0.6,2,1.7,1.6,1.5],[1.6,0,1.9,3.5,0.7,1.3,3.1],[0.6,1.9,0,1.6,2.2,2.2,1.6],[2,3.5,1.6,0,3.7,3.6,1.4],[1.7,0.7,2.2,3.7,0,0.7,3.1],[1.6,1.3,2.2,3.6,0.7,0,2.7], [1.5,3.1,1.6,1.4,3.1,2.7,0]
full_mat = np.array(f)
print(full_mat)
print(full_mat.shape)
###Input 인수들  
# number of vehicle = 1
D = [1000,200,300,400,50,600,700] #Weekly demand of item i
C = [10,60,30,400,4000,200,40] #Unit cost of item i
Cr = 0.01 #Inventory holding cost (% of unit cost/week)
Cp = [0,0,0,0,0,0,0] #Setup cost to pick up item i
VVC = 12 #Variable vehicle cost ($/hour of travel time)
P = [[1],[2],[3],[4],[5],[6,7]] #Set of all items to be picked up from vendor k

[[0.  1.6 0.6 2.  1.7 1.6 1.5]
 [1.6 0.  1.9 3.5 0.7 1.3 3.1]
 [0.6 1.9 0.  1.6 2.2 2.2 1.6]
 [2.  3.5 1.6 0.  3.7 3.6 1.4]
 [1.7 0.7 2.2 3.7 0.  0.7 3.1]
 [1.6 1.3 2.2 3.6 0.7 0.  2.7]
 [1.5 3.1 1.6 1.4 3.1 2.7 0. ]]
(7, 7)


### Step 2  

In [5]:
#dt_min, dt_max 구하기
dt_min, dt_max = cal_dtmin_dtmax(len(full_mat), full_mat)
print(dt_min) 
print(dt_max)

[2.0, 3.2, 3.0, 1.4, 2.0, 3.0]
[3.2, 3.2, 4.0, 3.4, 3.2, 3.0]


In [6]:
# n_max, n_min값 구하기
n_max,n_min = cal_nmax_nmin(D, C, Cr, Cp, VVC, P, dt_min, dt_max)
print(n_max,n_min)  

[1.44, 1.25, 1.12, 6.9, 6.45, 4.53] [1.14, 1.25, 0.97, 4.43, 5.1, 4.53]


# From here, repeated by "Aplha" 
### Step 3 

In [7]:
#alpha=0일 때 n(k)값
alpha = 0
nk_0 = cal_nk(alpha, P, n_max, n_min)  
print(nk_0) 
#밑에서는 반올림 해서 사용함 

[1.14, 1.25, 0.97, 4.43, 5.1, 4.53]


### Step 4

In [8]:
#alpha = 0 일 때의 요일별 vendor 방문 스케쥴
alp = 0  
nk, Mon, Tue, Wed, Thu, Fri = create_schedule(alp, P, n_max, n_min)   
print(nk)  
print(Mon)
print(Tue)
print(Wed)  
print(Thu)  
print(Fri)

[1.0, 1.0, 1.0, 4.0, 5.0, 5.0]
[4, 5, 6]
[4, 5, 6]
[1, 2, 3, 5, 6]
[4, 5, 6]
[4, 5, 6]


## From here, repeated by "CW" on each day of week

### Step 5

In [9]:
# 요일별로 CW 돌려서 반 최적의 routes 뽑아내야함 
# full matrix에서 step4에서 나온 요일별 스케줄 이용해서 자른 다음에 half_mat으로 바꿔서 넣어줘야 
# to_half_mat_by_day에서 출발노드인 0번 추가해주어야  

schedules = [Mon, Tue, Wed, Thu, Fri] 
names = ["Mon","Tue","Wed","Thu","Fri"]
total_week_cost = 0

for day_idx, day in enumerate(schedules):
    
    half_mat_day = to_half_mat_by_day(full_mat, day)  
    
    print("{}'s must visit vendor from zero:{}".format(names[day_idx], day))  #list not changed! (by using deepcopy)
    _, t_mat, demand, is_constraint = initialize(len(half_mat_day), False) 
    net_saving_mat = calculate_net_saving(half_mat_day, len(half_mat_day))
    print(net_saving_mat)   
    print()
    ############
    print("!!!loop start")  
    print()
    
    count = 0  
    while(True):
        count += 1
        print("iteration:{}".format(count)) 
        t_mat, adjacency, net_saving_mat, cells= max_net_saving(half_mat_day, t_mat, net_saving_mat, demand, is_constraint,  12345)
        #print(t_mat) 
        #print(adjacency)  
        #print(net_saving_mat)
        routes = search_all_route(adjacency)  
        print(routes)  
        print()
        #total_cost = calculate_cost(routes, half_mat_day)  
        #print(total_cost)
        
        if len(cells) == 0 or len(routes) == 1:     
            routes = search_all_route(adjacency)
            ##day에 맞춰서 출력해줄라구 인덱스? 조정..
            tmp_day = deepcopy(day)
            tmp_day.insert(0,0) 
            real_routes = deepcopy(routes) 
            for route_idx, route in enumerate(routes):
                for i in range(len(route)):
                    real_routes[route_idx][i] = tmp_day[route[i]]
                    
                    
            print("{}'s local optimal route(index name):{}".format(names[day_idx], routes))
            print("{}'s local optimal route(real name):{}".format(names[day_idx], real_routes))
            total_cost = calculate_cost(routes, half_mat_day)
            print("{}'s local optimal total cost:{}".format(names[day_idx],total_cost))
            total_week_cost += total_cost  
        
            print("loop end!!!")
            print()
            break
            
print("total_week_cost:{}".format(total_week_cost))

Mon's must visit vendor from zero:[4, 5, 6]
[[-1.  -1.  -1.  -1. ]
 [-1.  -1.   2.6  0.1]
 [-1.  -1.  -1.   0.4]
 [-1.  -1.  -1.  -1. ]]

!!!loop start

iteration:1
[[0, 1, 2, 0], [0, 3, 0]]

iteration:2
[[0, 1, 2, 3, 0]]

Mon's local optimal route(index name):[[0, 1, 2, 3, 0]]
Mon's local optimal route(real name):[[0, 4, 5, 6, 0]]
Mon's local optimal total cost:6.6
loop end!!!

Tue's must visit vendor from zero:[4, 5, 6]
[[-1.  -1.  -1.  -1. ]
 [-1.  -1.   2.6  0.1]
 [-1.  -1.  -1.   0.4]
 [-1.  -1.  -1.  -1. ]]

!!!loop start

iteration:1
[[0, 1, 2, 0], [0, 3, 0]]

iteration:2
[[0, 1, 2, 3, 0]]

Tue's local optimal route(index name):[[0, 1, 2, 3, 0]]
Tue's local optimal route(real name):[[0, 4, 5, 6, 0]]
Tue's local optimal total cost:6.6
loop end!!!

Wed's must visit vendor from zero:[1, 2, 3, 5, 6]
[[-1.  -1.  -1.  -1.  -1.  -1. ]
 [-1.  -1.   0.3  0.1  1.9  0. ]
 [-1.  -1.  -1.   1.   0.   0.5]
 [-1.  -1.  -1.  -1.   0.   2.1]
 [-1.  -1.  -1.  -1.  -1.   0.4]
 [-1.  -1.  -1.  -1. 

In [10]:
### function_ver 
#특정한 alpha값에 대해서 그때의 요일별 best schedule을 리턴해줌 
def calculate_total_week_cost(Mon, Tue, Wed, Thu, Fri, full_mat):
# 요일별로 CW 돌려서 반 최적의 routes 뽑아내야함 
# full matrix에서 step4에서 나온 요일별 스케줄 이용해서 자른 다음에 half_mat으로 바꿔서 넣어줘야 
# to_half_mat_by_day에서 출발노드인 0번 추가해주어야  

    schedules = [Mon, Tue, Wed, Thu, Fri] 
    names = ["Mon","Tue","Wed","Thu","Fri"]
    total_week_cost = 0
    local_optimal_schedule = []

    for day_idx, day in enumerate(schedules):

        half_mat_day = to_half_mat_by_day(full_mat, day)  #from vendorpickup.py 
        #print("{}'s must visit vendor from zero:{}".format(names[day_idx], day))  #list not changed! (by using deepcopy)
        _, t_mat, demand, is_constraint = initialize(len(half_mat_day), False) #from clarkewright.py  
        net_saving_mat = calculate_net_saving(half_mat_day, len(half_mat_day)) #from clarkewright.py  

        while(True):
            t_mat, adjacency, net_saving_mat, cells= max_net_saving(half_mat_day, t_mat, net_saving_mat, demand, is_constraint,  12345)
            routes = search_all_route(adjacency) 
            if len(cells) == 0 or len(routes) == 1:  
                routes = search_all_route(adjacency) 
                #실제 vendor number에 맞게 조정 
                tmp_day = deepcopy(day)
                tmp_day.insert(0,0) 
                real_routes = deepcopy(routes) 
                for route_idx, route in enumerate(routes):
                    for i in range(len(route)):
                        real_routes[route_idx][i] = tmp_day[route[i]]
                
                local_optimal_schedule.append(real_routes)
                total_cost = calculate_cost(routes, half_mat_day)  
                total_week_cost += total_cost 
                break
            
    return total_week_cost, local_optimal_schedule

### STEP 6  

In [12]:
# 모든 경우의 수를 뽑기 위한 모듈 import
from itertools import permutations

In [11]:
# 가능한 모든 경로를 추출하는 함수 (방문 노드는 고정, 여러 개의 루트가 나왔을 때도 사용 가능)
def get_everycase(routes):
    everycase = []
    for i in range(len(routes)):
        a = routes[i][1:-1]
        b = list(permutations(a,len(a)))
        e, routes_cases,routes_everycase = [], [], []
        
        for j in range(len(b)):        
            e.append(list(b[j]))
            routes_cases.append([0] + e[j] + [0])
            
        if len(routes) == 1:
            routes_everycase = routes_cases
            return routes_everycase
            break
            
        if len(everycase) == 0:
            for k in range(len(routes_cases)):
                d = []
                d.append(routes_cases[k])
                everycase.append(d)
                
        elif len(routes) == 2: 
            for k in range(len(everycase)):
                for l in range(len(routes_cases)):
                    everycase[k].append(routes_cases[l])
                    d3 = deepcopy(everycase)
                    routes_everycase.append(d3[k])
                    everycase[k].pop()
        else:
            m = 2
            while(m < len(routes)):
                for k in range(len(everycase)):
                    for l in range(len(routes_cases)):
                        everycase[k].append(routes_cases[l])
                        d3 = deepcopy(everycase)
                        routes_everycase.append(d3[k])
                        everycase[k].pop()
                d4 = deepcopy(routes_everycase)
                everycase = d4 
                m += 1
                break
                
    return routes_everycase

In [13]:
#단일 루트일 때 확인
routes = [[0,1,2,3,0]] #한 요일에 대한 schedule 
routes_everycase = get_everycase(routes)
routes_everycase

[[0, 1, 2, 3, 0],
 [0, 1, 3, 2, 0],
 [0, 2, 1, 3, 0],
 [0, 2, 3, 1, 0],
 [0, 3, 1, 2, 0],
 [0, 3, 2, 1, 0]]

In [14]:
#이중 루트일 때 확인
routes = [[0,1,2,0],[0,3,4,0]]
routes_everycase = get_everycase(routes)
routes_everycase

[[[0, 1, 2, 0], [0, 3, 4, 0]],
 [[0, 1, 2, 0], [0, 4, 3, 0]],
 [[0, 2, 1, 0], [0, 3, 4, 0]],
 [[0, 2, 1, 0], [0, 4, 3, 0]]]

In [15]:
#삼중 루트일 때 확인 // N>3인 다중 루트일 때도 가능
routes = [[0,1,2,0],[0,3,4,0],[0,5,6,0]]
routes_everycase = get_everycase(routes)
routes_everycase

[[[0, 1, 2, 0], [0, 3, 4, 0], [0, 5, 6, 0]],
 [[0, 1, 2, 0], [0, 3, 4, 0], [0, 6, 5, 0]],
 [[0, 1, 2, 0], [0, 4, 3, 0], [0, 5, 6, 0]],
 [[0, 1, 2, 0], [0, 4, 3, 0], [0, 6, 5, 0]],
 [[0, 2, 1, 0], [0, 3, 4, 0], [0, 5, 6, 0]],
 [[0, 2, 1, 0], [0, 3, 4, 0], [0, 6, 5, 0]],
 [[0, 2, 1, 0], [0, 4, 3, 0], [0, 5, 6, 0]],
 [[0, 2, 1, 0], [0, 4, 3, 0], [0, 6, 5, 0]]]

In [16]:
#요일별 모든 exchange 경로(index값)과 half_mat_day를 받았을 때, 최적 경로와 비용 계산 
def calculate_bestcost(routes_everycase, half_mat_day):
    #route에 따른 cost 리턴하기?  
    
    #half matrix를 전체 matrix로 확장(인접하게)
    #각 노드들 간의 최단 거리 행렬 
    changed_half_mat = np.copy(half_mat_day)
    for i in range(len(changed_half_mat)):
        for j in range(len(changed_half_mat)):
            if changed_half_mat[i,j] == -1:
                changed_half_mat[i,j] = 0
    
    #print(changed_half_mat)
    
    total_mat = changed_half_mat + np.transpose(changed_half_mat) 
    #print(total_mat)
    #routes에 따라서 cost 계산
    total_cost = 0
    best_cost = 10000
    
    for route_idx, route in enumerate(routes_everycase):
        for i in range(len(route)-1):  
            total_cost += total_mat[route[i],route[i+1]]
        total_cost = round(total_cost,10)
        print(f'{routes_everycase[route_idx]}일 때 비용 {total_cost}')
        if best_cost > total_cost :
            best_cost = total_cost
            best_route = routes_everycase[route_idx]
        total_cost = 0
    print(f' 최적 경로는 : {best_route}. 이 때의 비용은 : {best_cost}')     
    return best_cost, best_route

In [17]:
#기존 vendorpickup에 있는 함수
def to_half_mat_by_day(full_mat, day):    
    #요일별로 full matrix 잘라서 half_matrix 형태로 바꿔주기  
    # ex) half_mat = np.array([[-1,10,12,15,7],[-1,-1,5,12,11],[-1,-1,-1,7,9],[-1,-1,-1,-1,10],[-1,-1,-1,-1,-1]])
    
   
    #cut by indices
    indices = deepcopy(day) 
    indices.insert(0,0)
    indices = np.array(indices)
    #print(indices)
    tmp_mat = np.copy(full_mat[indices]) #cut row
    tmp_mat = np.copy(tmp_mat[:,indices]) #cut column
    
    #change to half matrix 
    half_mat = np.triu(tmp_mat)
    #print(half_mat)     
    for i in range(len(half_mat)):
        for j in range(len(half_mat)):
            if half_mat[i,j] == 0: #tmp_mat의 upper diagonal이 모두 0이 아닐 경우에만 이게 맞는 코딩임  
                half_mat[i,j] = -1
    #print(half_mat)

    
    return half_mat

In [1]:
#미구현
#요일별 EXCHANGE에 따른 경로 및 비용계산
def cal_exchange(days):
    routes = [[[0,1,2,3,0]],[[0,1,2,3,0]],[[0,1,4,5,3,2,0]],[[0,1,2,3,0]],[[0,1,2,3,0]]
    names = ["Mon","Tue","Wed","Thu","Fri"]  
              
    for day_idx, day in enumerate(routes):
        half_mat_day = to_half_mat_by_day(full_mat, day)
        routes_everycase = get_everycase(routes[day_idx])
        print(day)
        calculate_bestcost(routes_everycase, half_mat_day)  

SyntaxError: invalid syntax (<ipython-input-1-5370be05a356>, line 5)

In [18]:
#월화목금 계산을 위한 준비
routes = [[0,1,2,3,0]]
routes_everycase = get_everycase(routes)
half_mat_day = to_half_mat_by_day(full_mat, Mon)
print(routes_everycase) 
print(half_mat_day)

[[0, 1, 2, 3, 0], [0, 1, 3, 2, 0], [0, 2, 1, 3, 0], [0, 2, 3, 1, 0], [0, 3, 1, 2, 0], [0, 3, 2, 1, 0]]
[[-1.   1.7  1.6  1.5]
 [-1.  -1.   0.7  3.1]
 [-1.  -1.  -1.   2.7]
 [-1.  -1.  -1.  -1. ]]


In [19]:
#월화목금 최적 경로 및 비용 
MTTF, mttf= calculate_bestcost(routes_everycase, half_mat_day)
print(MTTF)  
print(mttf)

[0, 1, 2, 3, 0]일 때 비용 6.6
[0, 1, 3, 2, 0]일 때 비용 9.1
[0, 2, 1, 3, 0]일 때 비용 6.9
[0, 2, 3, 1, 0]일 때 비용 9.1
[0, 3, 1, 2, 0]일 때 비용 6.9
[0, 3, 2, 1, 0]일 때 비용 6.6
 최적 경로는 : [0, 1, 2, 3, 0]. 이 때의 비용은 : 6.6
6.6
[0, 1, 2, 3, 0]


In [448]:
#수요일 계산을 위한 준비 #half_mat_day 정상 작동 X
routes = [[0,1,4,5,3,2,0]]
routes_everycase = get_everycase(routes)
half_mat_day = to_half_mat_by_day(full_mat,Wed)
print(half_mat_day)

[[-1.   1.6  0.6  2.   1.7  1.6  1.5]
 [-1.  -1.   1.9  3.5  0.7  1.3  3.1]
 [-1.  -1.  -1.   1.6  2.2  2.2  1.6]
 [-1.  -1.  -1.  -1.   3.7  3.6  1.4]
 [-1.  -1.  -1.  -1.  -1.   0.7  3.1]
 [-1.  -1.  -1.  -1.  -1.  -1.   2.7]
 [-1.  -1.  -1.  -1.  -1.  -1.  -1. ]]


### Step 7 & 8   


##### aplha 값을 다르게 해서 step3부터 step 5까지 반복

- 무엇을 비교? 10개의 alpha 중 total_week_cost가 가장 작은 경우 반환 

In [9]:
mincost = 1000000 

best_schedule = None 

for alpha in range(11):   
    #step 3 & 4  
    nk, Mon, Tue, Wed, Thu, Fri = create_schedule(alpha/10, P, n_max, n_min)  
    #step 5 
    travel_cost, schedule = calculate_total_week_cost(Mon, Tue, Wed, Thu, Fri, full_mat)
    print("total travel cost when alpha:{} is {}".format(alpha/10, travel_cost))
    if travel_cost < mincost:  
        mincost = travel_cost
        best_schedule = schedule  
        
#print(best_schedule)   
names = ["Mon","Tue","Wed","Thu","Fri"]
for i in range(0,len(best_schedule)): 
    print("{}'s best schedule:{}".format(names[i], best_schedule[i]))     

total travel cost when alpha:0.0 is 35.6
total travel cost when alpha:0.1 is 35.7
total travel cost when alpha:0.2 is 35.7
total travel cost when alpha:0.3 is 35.7
total travel cost when alpha:0.4 is 35.7
total travel cost when alpha:0.5 is 35.7
total travel cost when alpha:0.6 is 35.7
total travel cost when alpha:0.7 is 35.7
total travel cost when alpha:0.8 is 35.7
total travel cost when alpha:0.9 is 35.7
total travel cost when alpha:1.0 is 35.7
Mon's best schedule:[[0, 4, 5, 6, 0]]
Tue's best schedule:[[0, 4, 5, 6, 0]]
Wed's best schedule:[[0, 1, 5, 6, 3, 2, 0]]
Thu's best schedule:[[0, 4, 5, 6, 0]]
Fri's best schedule:[[0, 4, 5, 6, 0]]
